# Anomaly Detection

In [1]:
import  pandas as pd
import numpy as np
import scipy.stats as ss

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
data = pd.read_excel(r"D:\Analytics\Undercutting\Sample data\Anomaly detection 2.xlsx")

In [9]:
data.shape

(13353, 30)

In [4]:
data.head(2)

,Distribution Channel,Division,Sold-To Party,Customer Group 3,Price List Type,Region Order,Plant,Material,Date,New date,...,CredMemQty,CredMemQty.1,Gross,Gross.1,Sales cost,Net,Net.1,RtnsQty,RtnsQty.1,Rtns (cst)
0,GT General Trade,PC Personal Care,2000001 N.CT. AGENCIES,006 GT,02 Super Stockist,TN TN,"TND1 CavinKare Pvt. Ltd, CHE DEPOT",KH0050HWP02R KARTHIKA HERBAL POWDER 50GM...,22.04.2019,2019-04-22,...,0.0,CV,21.2,KG,1406.63,18.0,KG,0.0,CV,0.0
1,GT General Trade,PC Personal Care,2000001 N.CT. AGENCIES,006 GT,02 Super Stockist,TN TN,"TND1 CavinKare Pvt. Ltd, CHE DEPOT",KH0050HWP02R KARTHIKA HERBAL POWDER 50GM...,07.05.2019,2019-05-07,...,0.0,CV,21.2,KG,1422.78,18.0,KG,0.0,CV,0.0


In [5]:
data.columns

Index(['Distribution Channel', 'Division', 'Sold-To Party', 'Customer Group 3',
       'Price List Type', 'Region Order', 'Plant', 'Material', 'Date',
       'New date', 'Sales', 'Returns', 'Cred.Memos', 'CM: net 1', 'Net Value',
       'Tax', 'Bill. Qty', 'Bill. Qty.1', 'CredMemQty', 'CredMemQty.1',
       'Gross', 'Gross.1', 'Sales cost', 'Net', 'Net.1', 'RtnsQty',
       'RtnsQty.1', 'Rtns (cst)'],
      dtype='object')

In [6]:
temp                   =  data["Material"].str.split("       ",expand=True)
data["Prod_Code"]      = temp[0]
data["Prod_Name"]      = temp[1]
data["Date"]           = pd.to_datetime(data["Date"],format='%d.%m.%Y')
# data["Month"]          = data["New date"].dt.strftime('%B')

In [32]:
dist_data = data[["Sold-To Party","Date","Prod_Name","Bill. Qty"]]

In [33]:
dist_data.shape

(13353, 4)

In [34]:
dist_data.head(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty
0,2000001 N.CT. AGENCIES,2019-04-22,KARTHIKA HERBAL POWDER 50GM 180 PCS,2.0
1,2000001 N.CT. AGENCIES,2019-05-07,KARTHIKA HERBAL POWDER 50GM 180 PCS,2.0


In [11]:
rmv_single_rows = dist_data.groupby(["Sold-To Party","Prod_Name"]).agg(Count = ("Prod_Name","count")).reset_index()
rmv_single_rows.head(2)

,Sold-To Party,Prod_Name,Count
0,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,31
1,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 50GM 180 PCS,10


In [12]:
len(rmv_single_rows[rmv_single_rows["Count"] == 1])

387

In [13]:
dist_data_clean = pd.merge(dist_data,rmv_single_rows,on = ["Sold-To Party","Prod_Name"],how = "left")
dist_data_clean = dist_data_clean[dist_data_clean["Count"] != 1]
dist_data_clean.shape

(12966, 5)

In [24]:
print("So totally {a} rows will be dropped from total of {b} rows. Because it has only 1 data".format(a = len(rmv_single_rows[rmv_single_rows["Count"] == 1]), b= len(dist_data)))

So totally 387 rows will be dropped from total of 13353 rows. Because it has only 1 data


In [14]:
dist_data_clean.head(2)

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count
0,2000001 N.CT. AGENCIES,2019-04-22,KARTHIKA HERBAL POWDER 50GM 180 PCS,2.0,10
1,2000001 N.CT. AGENCIES,2019-05-07,KARTHIKA HERBAL POWDER 50GM 180 PCS,2.0,10


In [39]:
# dist_data_groupby1.to_excel(r"D:\Analytics\Undercutting\Sample data\Outfiles\dist_data_groupby1.xlsx",index = False)

In [41]:
dist_data_descriptive = dist_data_clean.groupby(["Sold-To Party","Prod_Name"]).agg(Mean = ("Bill. Qty","mean"), 
                                           stddev = ("Bill. Qty","std"),Median =("Bill. Qty","median")).reset_index()
dist_data_descriptive.isna().sum()

Sold-To Party    0
Prod_Name        0
Mean             0
stddev           0
Median           0
dtype: int64

In [42]:
zscore_data = pd.merge(dist_data_clean, dist_data_descriptive, on = ["Sold-To Party","Prod_Name"], how = 'left')

In [43]:
# Test Data
zscore_data[(zscore_data["Sold-To Party"] == "2000001    N.CT. AGENCIES") & (zscore_data["Prod_Name"] == "KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS")].head(2)

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median
61,2000001 N.CT. AGENCIES,2019-04-04,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,118.0,31,232.322581,233.914427,192.0
63,2000001 N.CT. AGENCIES,2019-04-16,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,160.0,31,232.322581,233.914427,192.0


In [44]:
zscore_data.isna().sum()
# zscore_data.dropna(axis = 0, inplace = True)

Sold-To Party    0
New date         0
Prod_Name        0
Bill. Qty        0
Count            0
Mean             0
stddev           0
Median           0
dtype: int64

# Z Score Calculation

In [45]:
zscore_data['z-score']= (zscore_data['Bill. Qty'] - zscore_data['Mean'])/zscore_data['stddev']

In [46]:
# Test Data
zscore_data[(zscore_data["Sold-To Party"] == "2000001    N.CT. AGENCIES") & (zscore_data["Prod_Name"] == "KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS")].tail(2)

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score
121,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,31,232.322581,233.914427,192.0,4.564393
122,2000001 N.CT. AGENCIES,2019-09-30,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,439.0,31,232.322581,233.914427,192.0,0.883560


In [47]:
zscore_data.sort_values(by='z-score', ascending=False,inplace=True)
zscore_data.head(5)

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score
121,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,31,232.322581,233.914427,192.0,4.564393
11895,2003538 EVERGREEN ENTERPRISES,2019-09-17,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,130.0,22,20.000000,25.266202,15.0,4.353642
11702,2003538 EVERGREEN ENTERPRISES,2019-07-20,KARTHIKA HERBAL POWDER 50GM 180 PCS,7.0,31,1.903226,1.247794,2.0,4.084629
2873,2000090 SRI SRINIVASA AGENCIES,2019-09-27,KARTHIKA HERBAL POWDER CONT 180G 80PCS,8.0,21,1.714286,1.553797,1.0,4.045389
5511,2000125 SRI AMMAN AGENCY,2019-09-23,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,49.0,24,13.458333,8.997484,12.5,3.950179


In [48]:
zscore_data.head(5)

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score
121,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,31,232.322581,233.914427,192.0,4.564393
11895,2003538 EVERGREEN ENTERPRISES,2019-09-17,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,130.0,22,20.000000,25.266202,15.0,4.353642
11702,2003538 EVERGREEN ENTERPRISES,2019-07-20,KARTHIKA HERBAL POWDER 50GM 180 PCS,7.0,31,1.903226,1.247794,2.0,4.084629
2873,2000090 SRI SRINIVASA AGENCIES,2019-09-27,KARTHIKA HERBAL POWDER CONT 180G 80PCS,8.0,21,1.714286,1.553797,1.0,4.045389
5511,2000125 SRI AMMAN AGENCY,2019-09-23,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,49.0,24,13.458333,8.997484,12.5,3.950179


# Mod Z Score calculation

In [49]:
zscore_data['MAD Bill. Qty'] = np.absolute(zscore_data['Bill. Qty'] - zscore_data['Median'])

In [50]:
zscore_data['Mod Z score'] = 0.6745 * zscore_data['MAD Bill. Qty']/zscore_data['Median']

In [51]:
zscore_data.sort_values(by=['Sold-To Party','Prod_Name'], ascending=False,inplace=True)
zscore_data

,Sold-To Party,New date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score,MAD Bill. Qty,Mod Z score
12964,2007255 SRI NARASIMHA AGENCIES,2019-08-31,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,2,1.000000,1.414214,1.0,0.707107,1.0,0.674500
12965,2007255 SRI NARASIMHA AGENCIES,2019-09-18,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,0.0,2,1.000000,1.414214,1.0,-0.707107,1.0,0.674500
12956,2007226 ADITHYAA ENTERPRISES,2019-09-23,MEERA ADVANCE HW POWDER 80GM REFILL 50 P,5.0,4,2.500000,1.914854,2.0,1.305582,3.0,1.011750
12955,2007226 ADITHYAA ENTERPRISES,2019-09-17,MEERA ADVANCE HW POWDER 80GM REFILL 50 P,3.0,4,2.500000,1.914854,2.0,0.261116,1.0,0.337250
12953,2007226 ADITHYAA ENTERPRISES,2019-08-28,MEERA ADVANCE HW POWDER 80GM REFILL 50 P,1.0,4,2.500000,1.914854,2.0,-0.783349,1.0,0.337250
...,...,...,...,...,...,...,...,...,...,...,...
109,2000001 N.CT. AGENCIES,2019-08-14,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,50.0,31,232.322581,233.914427,192.0,-0.779441,142.0,0.498849
100,2000001 N.CT. AGENCIES,2019-07-23,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,40.0,31,232.322581,233.914427,192.0,-0.822192,152.0,0.533979
104,2000001 N.CT. AGENCIES,2019-08-05,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,40.0,31,232.322581,233.914427,192.0,-0.822192,152.0,0.533979
120,2000001 N.CT. AGENCIES,2019-09-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,30.0,31,232.322581,233.914427,192.0,-0.864943,162.0,0.569109


In [54]:
zscore_data["z-score"].fillna(0,inplace=True)

In [56]:
zscore_data.to_excel(r"D:\Analytics\Undercutting\Sample data\Dashboard\Z-score data.xlsx",index = False)